# Import libraries

In [ ]:
!pip install numpy
!pip install scikit-surprise

In [1]:
import pandas as pd
import streamlit as st
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
from surprise import KNNBasic, SVD, KNNWithMeans, SVDpp
from surprise import accuracy
from surprise.model_selection import GridSearchCV

# Cleaning

In [3]:
%run cleaning.ipynb

Ratings table has 1149780 rows and 3 columns
Books table has 271360 rows and 8 columns
Users table has 278858 rows and 3 columns
Columns and their data types:
book_title             object
book_author            object
year_of_publication    object
publisher              object
image_url_m            object
common_identifier       int64
dtype: object
Books table has 239094 rows and 6 columns
Columns and their data types:
user_id         int64
isbn           object
book_rating     int64
dtype: object

Number of duplicated rows:0

Number of rows with missing values:0
Ratings table has 1149780 rows and 4 columns

There are 118646 rows in ratings_df that do not have a common identifier in books_df
Columns and their data types:
user_id     int64
age         int64
city       object
country    object
dtype: object
Users table has 275565 rows and 4 columns
There are 0 missing values in df
There are 0 duplicates in df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005487 entries, 0 to 10054

# Load Data

In [4]:
# Read data
df = pd.read_csv('data/kaggle_full_df.csv')
df['country'].fillna('unknown', inplace=True)

In [63]:
df.shape

(1005487, 12)

# Baseline Model (Popularity Model)

A common (and usually hard-to-beat) baseline approach is the Popularity model. This model is not actually personalized - it simply recommends to a user the most popular items that the user has not previously consumed. As the popularity accounts for the "wisdom of the crowds", it usually provides good recommendations, generally interesting for most people. The main objective of a recommender system is to leverage the long-tail items to the users with very specific interests, which goes far beyond this simple technique.

In [ ]:
# Recommend to every user the five books with the highest mean rating
top_5_books = df.groupby('common_identifier')['book_rating'].mean().sort_values(ascending=False).head(5)
top_5_books

In [ ]:
# Get the common identifiers of the top 5 books
top_5_common_identifiers = top_5_books.index

# Filter the original DataFrame to get the corresponding book titles
top_5_books_info = df[df['common_identifier'].isin(top_5_common_identifiers)][['common_identifier', 'book_title']].drop_duplicates()

# Display the top 5 books by name
print(top_5_books_info)

In [1]:
# Get recommendations for a user
def get_recommendations(user_id, num_books=5):
    # Get the books the user has rated
    user_rated_books = df[df['user_id'] == user_id][['common_identifier', 'book_rating']]
    # Get the books the user has not rated
    user_unrated_books = df[~df['common_identifier'].isin(user_rated_books['common_identifier'])][['common_identifier', 'book_rating']]
    # Merge user_unrated_books with top_5_books_info to get the book titles
    recommendations = pd.merge(user_unrated_books, top_5_books_info, on='common_identifier', how='inner')
    # Sort the recommendations by rating in descending order
    recommendations = recommendations.sort_values(by='book_rating', ascending=False)
    # Return the top num_books recommendations
    return recommendations.head(num_books)

# Difference between User-based and Item-based Collaborative Filtering

- User-based collaborative filtering recommends items to a user based on the preferences of similar users. It finds users who have similar tastes and preferences to the target user and recommends items that those similar users have liked or rated highly. For a given user, the algorithm looks for other users who have rated or liked similar items and calculates the similarity between the target user and those similar users. The algorithm then predicts the ratings or preferences of the target user for items that they have not yet interacted with, based on the rating or preferences of the similar users. User-based collaborative filtering is more suitable when the user-item matrix is sparse and there are more users than items.

- Item-based collaborative filtering recommends items to a user based on the similarity between items. It finds items that are similar to the ones the target user has already interacted with and recommends those similar items. For a given item, the algorithm looks for other items that have been rated or liked by users who have also rated or liked the target item. The algorithm calculates the similarity between the target item and those similar items based on the rating or preferences of the users who have interacted with both items. The algorithm then predicts the ratings or preferences of the target user for items that are similar to the ones they have already interacted with. Item-based collaborative filtering is more suitable when the user-item matrix is dense and there are more items than users.

- In summary, user-based filtering focuses on finding similar users to make recommendations, while item-based collaborative filtering focuses on finding similar items. The choice between the two depens on the characteristics of the dataset and the specific requirements of the recommendation system.

# Modelling

## Subset data

In [5]:
#Only Rating above 0
df = df[df['book_rating']>0]

#Only users from US or Canada
df = df[df['country'].str.contains("usa|canada")]

df.shape

(303032, 13)

#### Create a new variable: Rating Count

In [6]:
#Add a new column with a total rating count for each book by common identifier
df['rating_count'] = df.groupby(['book_title', 'book_author'])['book_rating'].transform('count')

#Show a list of books that got the highest rating count, group by title and author to show unique books

df.groupby(['book_title', 'book_author', 'rating_count']).size().reset_index(name='Count').sort_values(by='rating_count', ascending=False).head(5)

,book_title,book_author,rating_count,Count
82908,The Lovely Bones: A Novel,Alice Sebold,614.00,614
33162,Harry Potter And The Sorcerer'S Stone,J. K. Rowling,445.00,445
76421,The Da Vinci Code,Dan Brown,421.00,421
65060,She'S Come Undone,Wally Lamb,411.00,411
87958,The Secret Life Of Bees,Sue Monk Kidd,387.00,387


In [7]:
df.head(3)

,book_title,book_author,year_of_publication,publisher,image_url_m,common_identifier,user_id,isbn,book_rating,age,city,country,description,rating_count
1,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,1,269782,0801319536,7,30,edmonton,canada,NaN,1.00
2,Pay It Forward: A Novel,Catherine Ryan Hyde,2000,Simon &amp; Schuster,http://images.amazon.com/images/P/0684862719.0...,2392,269782,0684862719,8,30,edmonton,canada,is a wondrous and moving story about Trevor Mc...,26.00
3,Watership Down,Richard Adams,1976,Avon,http://images.amazon.com/images/P/0380002930.0...,3172,269782,0140039589,10,30,edmonton,canada,"Set in England's Downs, a once idyllic rural l...",101.00


#### Define user activity threshold

In [8]:
#Subset only users with more than 30 ratings

user_rating_counts = df['user_id'].value_counts()
df = df[df['user_id'].isin(user_rating_counts[user_rating_counts >= 30].index)]
df.shape

(151974, 14)

In [9]:
popularity_threshold = 50
df = df[df['rating_count'] >= popularity_threshold]
df.shape

(21365, 14)

In [11]:
#Add a new column with a total rating count for each book by common identifier
df['rating_count'] = df.groupby(['book_title', 'book_author'])['book_rating'].transform('count')

#Show a list of books that got the highest rating count, group by title and author to show unique books

df.groupby(['book_title', 'book_author', 'rating_count']).size().reset_index(name='Count').sort_values(by='rating_count', ascending=False).head(5)

,book_title,book_author,rating_count,Count
193,Harry Potter And The Sorcerer'S Stone,J. K. Rowling,194,194
459,The Lovely Bones: A Novel,Alice Sebold,165,165
189,Harry Potter And The Chamber Of Secrets,J. K. Rowling,144,144
334,She'S Come Undone,Wally Lamb,133,133
401,The Da Vinci Code,Dan Brown,133,133


In [12]:
df.head(3)

,book_title,book_author,year_of_publication,publisher,image_url_m,common_identifier,user_id,isbn,book_rating,age,city,country,description,rating_count
110,To Kill A Mockingbird,Harper Lee,1988,Little Brown &amp; Company,http://images.amazon.com/images/P/0446310786.0...,38,85526,0060935464,9,36,victoria,canada,The unforgettable novel of a childhood in a sl...,101
111,Pride And Prejudice,Jane Austen,1983,Bantam,http://images.amazon.com/images/P/055321215X.0...,42,85526,019282760X,8,36,victoria,canada,"Since its immediate success in 1813, Pride and...",67
118,Girl With A Pearl Earring,Tracy Chevalier,2001,Plume Books,http://images.amazon.com/images/P/0452282152.0...,232,85526,0452282152,7,36,victoria,canada,"With precisely 35 canvases to his credit, the ...",77


## Build Models

In [13]:
# Define rating scale
reader = Reader(rating_scale=(1, 10))
# Load data into Surprise dataset
data = Dataset.load_from_df(df[['user_id', 'common_identifier', 'book_rating']], reader)
#data_all = Dataset.load_from_df(df[['user_id', 'common_identifier', 'book_rating']], reader)

In [14]:
# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
#trainset_all, testset_all = train_test_split(data_all, test_size=0.2, random_state=42)

## SVD

In [15]:
# Build SVD model 
svd_model = SVD()
svd_model.fit(trainset)

In [16]:
# Evaluation of SVD model
svd_predictions = svd_model.test(testset)
svd_rmse = accuracy.rmse(svd_predictions)
# RMSE: 1.5334

RMSE: 1.5226


In [17]:
# Perform cross validation
svd_cv = cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Print Mean RMSE Score
print("Mean RMSE Score: ", svd_cv['test_rmse'].mean())
# Mean RMSE Score:  1.5288

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5318  1.5273  1.5296  1.5063  1.5032  1.5196  0.0123  
MAE (testset)     1.1697  1.1652  1.1711  1.1405  1.1426  1.1578  0.0134  
Fit time          0.09    0.07    0.07    0.07    0.08    0.08    0.01    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    
Mean RMSE Score:  1.5196371086755918


## SVDpp

In [19]:
# Build SVDpp model 
## The SVD++ algorithm, an extension of SVD taking into account implicit ratings.
svdpp_model = SVDpp()
svdpp_model.fit(trainset)


In [20]:
# Evaluation of svdpp_predictions = svdpp_model.test(testset)
svdpp_rmse = accuracy.rmse(svdpp_predictions)SVDpp model
# RMSE: 1.5474

RMSE: 1.5474


In [88]:
# Perform cross validation
svdpp_cv = cross_validate(svdpp_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Print Mean RMSE Score
print("Mean RMSE Score: ", svdpp_cv['test_rmse'].mean())
# Mean RMSE Score:  1.5223

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5378  1.5189  1.5402  1.5168  1.4976  1.5223  0.0156  
MAE (testset)     1.1826  1.1789  1.1926  1.1675  1.1547  1.1753  0.0131  
Fit time          0.13    0.10    0.10    0.10    0.10    0.11    0.01    
Test time         0.04    0.04    0.04    0.04    0.04    0.04    0.00    
Mean RMSE Score:  1.5222709761619835


## KNN

In [95]:
# Build KNN model with cross-validation
knn_model = KNNWithMeans()
knn_model.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [96]:
# Evaluation of KNN model
knn_predictions = knn_model.test(testset)
knn_rmse = accuracy.rmse(knn_predictions)

RMSE: 1.6465


In [97]:
# Perform cross validation
knn_cv = cross_validate(knn_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# Print Mean RMSE Score
print("Mean RMSE Score: ", knn_cv['test_rmse'].mean())
# Mean RMSE Score:  1.6157

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5780  1.6221  1.6419  1.6110  1.6256  1.6157  0.0213  
MAE (testset)     1.1705  1.2040  1.2115  1.2008  1.2038  1.1981  0.0142  
Fit time          0.04    0.02    0.02    0.02    0.02    0.03    0.01    
Test time         0.06    0.06    0.06    0.06    0.06    0.06    0.00    
Mean RMSE Score:  1.6156944605093642


RMSE measures the average difference between the predicted ratings and the actual ratings given by users. A lower RMSE value indicates that the model's predictions are closer to the actual ratings, indicating better accuracy.

# Tuning

## SVD

In [113]:
param_grid = {
    "n_epochs": [5, 10, 15, 20], # number of iterations
    "lr_all": [0.002, 0.005, 0.01, 0.02], # learning rate for all parameters, which decides how much the parameters are adjusted in each iteration
    "reg_all": [0.4, 0.6, 0.8, 1.0]# regularization term for all parameters, which is a penalty term added to prevent overfitting
}

gs = GridSearchCV(SVD, param_grid, measures=["rmse"], cv=5)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

# RMSE: 1.5217

1.519450252229476
{'n_epochs': 20, 'lr_all': 0.02, 'reg_all': 0.4}


In [114]:
# Build SVD model with best parameters
svd_model = SVD(n_epochs=20, lr_all=0.02, reg_all=0.4)
svd_model.fit(trainset)
svd_predictions = svd_model.test(testset)
svd_rmse = accuracy.rmse(svd_predictions)

# Perform cross validation with best parameters
svd_cv = cross_validate(svd_model, data, measures=['rmse'], cv=5, verbose=True)

# Print Mean RMSE Score
print("Mean RMSE Score: ", svd_cv['test_rmse'].mean())

RMSE: 1.5307
Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4987  1.5296  1.5327  1.5161  1.5367  1.5227  0.0139  
Fit time          0.06    0.06    0.06    0.06    0.06    0.06    0.00    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    
Mean RMSE Score:  1.5227479762204075


## KNN

In [115]:
# grid search for KNN
param_grid = {
    "k": [10, 20, 30, 40, 50, 100], # number of neighbors
    "sim_options": {
        "name": ["msd", "cosine"], # similarity measure
        "min_support": [1, 2, 3, 4, 5, 10, 20], # minimum number of common items for two users
        "user_based": [True, False] # user-based or item-based
    }
}
gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse"], cv=5)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [116]:
# Build KNN with best parameters
knn_model = KNNWithMeans(k=100, sim_options={'name': 'cosine', 'min_support': 1, 'user_based': False})
knn_model.fit(trainset)
knn_predictions = knn_model.test(testset)
knn_rmse = accuracy.rmse(knn_predictions)

# Perform cross validation with best parameters
knn_cv = cross_validate(knn_model, data, measures=['rmse'], cv=5, verbose=True) 

# Print Mean RMSE Score
print("Mean RMSE Score: ", knn_cv['test_rmse'].mean())


Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.5897
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5731  1.5299  1.5469  1.6033  1.6140  1.5734  0.0321  
Fit time          0.01    0.01    0.01    0.01    0.01    0.01    0.00    
Test time         0.03    0.03    0.03    0.03    0.03    0.03    0.00    
Mean RMSE Score:  1.5734332509482976


## SVDpp

In [117]:
param_grid = {
    'n_epochs': [5, 10, 15],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.4, 0.6, 0.8]
}

gs = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=5)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])


1.5279679467634113
{'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.4}


In [119]:
# Build SVDpp model with best parameters
## The SVD++ algorithm, an extension of SVD taking into account implicit ratings.
svdpp_model = SVDpp(n_epochs=15, lr_all=0.01, reg_all=0.4)
svdpp_model.fit(trainset)
svdpp_predictions = svdpp_model.test(testset)
svdpp_rmse = accuracy.rmse(svdpp_predictions)

# Perform cross validation with best parameters
svdpp_cv = cross_validate(svdpp_model, data, measures=['rmse'], cv=5, verbose=True) 

# Print Mean RMSE Score
print("Mean RMSE Score: ", svdpp_cv['test_rmse'].mean())


RMSE: 1.5313
Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4901  1.5233  1.5560  1.5196  1.5442  1.5267  0.0226  
Fit time          0.10    0.10    0.11    0.10    0.12    0.11    0.01    
Test time         0.04    0.04    0.04    0.04    0.05    0.04    0.00    
Mean RMSE Score:  1.5266607544081967


# Recommendations

In [18]:
def get_svd_recommendations(user_id, svd_model, num_books=5):
    # Get the books the user has rated
    user_rated_books = df[df['user_id'] == user_id][['common_identifier', 'book_title', 'book_author']]
    
    # Get the books the user has not rated
    user_unrated_books = df[~df['common_identifier'].isin(user_rated_books['common_identifier'])][['common_identifier', 'book_title', 'book_author']]
    
    # Make predictions for unrated books using SVD model
    unrated_books = list(user_unrated_books['common_identifier'])
    user_predictions = [
        (book_id, svd_model.predict(user_id, book_id).est) for book_id in unrated_books
    ]
    
    # Remove duplicate entries based on common_identifier
    user_predictions = list(set(user_predictions))
    
    # Sort the predictions by estimated rating in descending order
    user_predictions.sort(key=lambda x: x[1], reverse=True)
    
    # Take the top num_books predictions
    user_predictions = user_predictions[:num_books]
    
    # Create a DataFrame from the sorted predictions
    recommendations = pd.DataFrame(user_predictions, columns=['common_identifier', 'estimated_rating'])
    
    # Merge with book information to get the book titles and authors
    recommendations = pd.merge(
        recommendations,
        user_unrated_books[['common_identifier', 'book_title', 'book_author']],
        on='common_identifier',
        how='inner'
    )
    
    # Drop duplicates based on common_identifier
    recommendations = recommendations.drop_duplicates(subset=['common_identifier'])
    
    # Drop the book_rating column
    recommendations = recommendations[['common_identifier', 'estimated_rating', 'book_title', 'book_author']]
    
    return recommendations.head(num_books)

# Replace 'user_id' with the actual user ID for which you want to get recommendations
user_id = 276729
recommended_books = get_svd_recommendations(user_id, svd_model, num_books=5)
print(recommended_books)

     common_identifier  estimated_rating  \
0                 3016              9.21   
29                 781              9.19   
59                4207              9.16   
98                6513              9.14   
140               3840              9.12   

                                   book_title       book_author  
0                              Atlas Shrugged          Ayn Rand  
29                             The Two Towers    J.R.R. Tolkien  
59                     The Return Of The King    J.R.R. Tolkien  
98                             The Two Towers  J. R. R. Tolkien  
140  Harry Potter And The Prisoner Of Azkaban     J. K. Rowling  


In [29]:
# Creating dataframe for unread books

unread_books = df[df['user_id'] != 276729]

In [30]:
user_df = df[df['user_id'] == 276729]

In [27]:
# Alternative method to get recommendations for a user
# Make predictions for one user using SVD model
one_user_predictions_svd = svd_model.predict(user_df, unread_books)
one_user_predictions_svd_flat = one_user_predictions_svd.est.flatten()

# Find the index of the highest predicted rating for unread books
index_of_highest_rating_unread_svd = one_user_predictions_svd_flat.argmax()

# Retrieve book information for the highest-rated book
recommended_book_title_unread_svd = unread_books['book_title'].values[index_of_highest_rating_unread_svd]
recommended_book_author_unread_svd = unread_books['book_author'].values[index_of_highest_rating_unread_svd]
recommended_book_genre_unread_svd = unread_books['genre'].values[index_of_highest_rating_unread_svd]
recommended_book_publication_unread_svd = unread_books['year_of_publication'].values[index_of_highest_rating_unread_svd]

# Display the recommendation for one user based on SVD model
print(f"You may like the book '{recommended_book_title_unread_svd}' from author {recommended_book_author_unread_svd} in the {recommended_book_genre_unread_svd} genre published in {recommended_book_publication_unread_svd}")

# Get indices of top 5 recommended books based on predicted ratings from SVD
top_5_indices_of_top_books_svd = one_user_predictions_svd_flat.argsort()[-5:][::-1]

# Retrieve information for the top 5 recommended books from SVD
top_5_books_one_user_svd = unread_books.iloc[top_5_indices_of_top_books_svd]

# Display the top 5 recommendations for one user based on SVD
print("Our top 5 book recommendations for you:")
displayed_titles_svd = set()  # keep track of displayed titles to remove duplicates
for i, book_info_svd in enumerate(top_5_books_one_user_svd.itertuples(), 1):
    if book_info_svd.book_title not in displayed_titles_svd: # remove duplicates
        # Print only if the book title is not already displayed
        print(f"{i}. '{book_info_svd.book_title}' from Author {book_info_svd.book_author} in the {book_info_svd.genre} genre, published in {book_info_svd.year_of_publication}")
        displayed_titles_svd.add(book_info_svd.book_title)

TypeError: unhashable type: 'DataFrame'

In [50]:
df['user_id'].head()

110    85526
111    85526
118    85526
126    85526
128    85526
Name: user_id, dtype: int64

In [59]:
# Extract user ID from user_df (replace 'user_id' with the actual column name)
user_id = 85526
# Extract item IDs (common_identifier) from unread_books (replace 'common_identifier' with the actual column name)
item_ids = unread_books['common_identifier'].values
# Make predictions for one user using SVD model
one_user_predictions_svd = [svd_model.predict(user_id, item_id) for item_id in item_ids]
one_user_predictions_svd
# Get indices of top 5 recommended books based on predicted ratings from SVD
top_5_indices_of_top_books_svd = sorted(range(len(one_user_predictions_svd)), key=lambda i: one_user_predictions_svd[i].est, reverse=True)[:5][::-1]

# Create a set to keep track of displayed common_identifiers
displayed_common_identifiers = set()

# Display the top 5 recommendations for one user based on SVD
print("Our top 5 book recommendations for you:")
#for i, index in enumerate(top_5_indices_of_top_books_svd):
for index in top_5_indices_of_top_books_svd:
    prediction = one_user_predictions_svd[index]
    book_info_svd = unread_books.iloc[index]
    common_identifier = book_info_svd.common_identifier
    if common_identifier not in displayed_common_identifiers:
        print(f"{i}. '{book_info_svd.book_title}' from Author {book_info_svd.book_author}, published in {book_info_svd.year_of_publication}, Estimated Rating: {prediction.est}")
        displayed_common_identifiers.add(common_identifier)


[Prediction(uid=85526, iid=38, r_ui=None, est=8.393779501991368, details={'was_impossible': False}),
 Prediction(uid=85526, iid=42, r_ui=None, est=7.908030022970034, details={'was_impossible': False}),
 Prediction(uid=85526, iid=232, r_ui=None, est=7.6859007693319095, details={'was_impossible': False}),
 Prediction(uid=85526, iid=480, r_ui=None, est=7.973570295605004, details={'was_impossible': False}),
 Prediction(uid=85526, iid=523, r_ui=None, est=8.10851282927116, details={'was_impossible': False}),
 Prediction(uid=85526, iid=566, r_ui=None, est=7.086467345762692, details={'was_impossible': False}),
 Prediction(uid=85526, iid=648, r_ui=None, est=7.912727974681387, details={'was_impossible': False}),
 Prediction(uid=85526, iid=715, r_ui=None, est=8.073697305096886, details={'was_impossible': False}),
 Prediction(uid=85526, iid=751, r_ui=None, est=6.4557346787265, details={'was_impossible': False}),
 Prediction(uid=85526, iid=1052, r_ui=None, est=7.270974923023575, details={'was_impos